In [20]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import csv
import sqlite3
import time

# Sukuriame naršyklės objektą
driver = webdriver.Chrome()

# Atidarome pagrindinį puslapį
url = "http://lemora.lt/262-sviestuvai/"
driver.get(url)
driver.maximize_window()

# Palaukiame, kad puslapis pilnai užsikrautų
time.sleep(2)

# Sutinkame su slapukais (jei reikia)
try:
    slapuku_mygtukas = WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.ID, "CybotCookiebotDialogBodyButtonDecline"))
    )
    slapuku_mygtukas.click()
    print("Pasirinkta: Tik būtini slapukai.")
except:
    print("Slapukų mygtukas nerastas arba jau priimtas.")

# Gauti puslapio HTML turinį
soup = BeautifulSoup(driver.page_source, "html.parser")

# Prisijungiame prie SQLite duomenų bazės
conn = sqlite3.connect("sviestuvai.db")
c = conn.cursor()

# Sukuriame lentelę, jei jos dar nėra
c.execute('''
    CREATE TABLE IF NOT EXISTS sviestuvai (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        pavadinimas TEXT UNIQUE,
        kaina TEXT,
        sviesos_srautas TEXT
    )
''')

# Surandame visus produktų blokus
produktai = soup.find_all("div", class_="product-container")

# Apdorojame visus produktus
for produktas in produktai:
    pavadinimas = produktas.find("a", class_="product-name").text.strip()
    kaina_elem = produktas.find("span", class_="price")
    kaina = kaina_elem.text.strip() if kaina_elem else "Nėra kainos"
    
    # Gauname produkto nuorodą ir atidarome jį naujame lange
    produkto_nuoroda = produktas.find("a", class_="product-name")["href"]
    driver.get(produkto_nuoroda)
    time.sleep(2)  # Palaukiame, kol užsikraus
    
    # Gauti produkto HTML
    produktas_soup = BeautifulSoup(driver.page_source, "html.parser")

    # Bandome rasti "Šviesos srautas" reikšmę
    sviesos_srautas = "Nerasta"
    aprasymas_elem = produktas_soup.find("div", class_="rte product-description")
    if aprasymas_elem:
        aprasymas_tekstas = aprasymas_elem.get_text(separator="\n")
        for eilute in aprasymas_tekstas.split("\n"):
            if "Šviesos srautas" in eilute:
                sviesos_srautas = eilute.split(":")[-1].strip()
                break

    # Grįžtame į pagrindinį puslapį
    driver.back()
    time.sleep(2)

    # **Atnaujiname arba įrašome į duomenų bazę**
    c.execute("SELECT id FROM sviestuvai WHERE pavadinimas = ?", (pavadinimas,))
    existing_product = c.fetchone()

    if existing_product:
        # Jei produktas jau yra, atnaujiname jo kainą ir šviesos srautą
        c.execute(
            "UPDATE sviestuvai SET kaina = ?, sviesos_srautas = ? WHERE pavadinimas = ?",
            (kaina, sviesos_srautas, pavadinimas),
        )
        print(f"Atnaujintas: {pavadinimas}")
    else:
        # Jei produkto nėra, įrašome jį į duomenų bazę
        c.execute(
            "INSERT INTO sviestuvai (pavadinimas, kaina, sviesos_srautas) VALUES (?, ?, ?)",
            (pavadinimas, kaina, sviesos_srautas),
        )
        print(f"Pridėtas: {pavadinimas}")

# Išsaugome pakeitimus ir uždarome ryšį
conn.commit()
conn.close()

print(f"Atnaujinta duomenų bazė (sviestuvai.db)")

# Uždaryti naršyklę
#driver.quit()


Pasirinkta: Tik būtini slapukai.
Atnaujintas: Šviestuvas vidaus SLV ONE TRIPLE LED, 64-65W, 4200lm, IP20 (įv.spalvos)
Atnaujintas: Šviestuvas lauko SLV PHOTONIA, pastatomas, E27, 60W, IP55, 90cm
Atnaujintas: Šviestuvas Northcliffe LEVANTO PLN LED, 36W, 3650lm, IP40
Atnaujintas: Šviestuvas Northcliffe CETUS P LED, sieninis-lubinis, 12W, 1080lm, IP65
Atnaujintas: Šviestuvas vidaus SLV PANTILO CONVEX, Ø29-40cm, E27, 40W, IP20
Atnaujintas: Šviestuvas Northcliffe DISC LED, 100-200W, 14000-28000lm, IP65
Atnaujintas: Šviestuvas lauko SLV ROTOBALL FLOOR, E27, 24W, IP44, 50cm
Atnaujintas: Toršeras lauko SLV ADEGAN MANILA SL, nešiojamas, E27, 15W, IP44, 175.5cm
Atnaujintas: Šviestuvas Northcliffe LIBRA LED, evakuacinis, sieninis, 1W, 6500K, IP65
Atnaujintas: Šviestuvas SLV VILUA UP/DOWN LED, fasadinis, 16W, 784lm, IP54
Atnaujintas: Prožektorius Northcliffe BETA LED, laukui, 30-200W, 2700-18000lm, IP65
Atnaujintas: Toršeras lauko SLV ENOLA C POLE LED, 12W, 800lm, IP55, 51cm
Atnaujintas: Troselių 

In [21]:
import sqlite3

# Prisijungiame prie SQLite duomenų bazės
conn = sqlite3.connect("sviestuvai.db")
c = conn.cursor()

# Patikriname, kiek įrašų yra duomenų bazėje
c.execute("SELECT COUNT(*) FROM sviestuvai")
kiekis = c.fetchone()[0]
print(f"Duomenų bazėje išsaugoti {kiekis} produktai.")

# Ištraukiame kelis pirmus įrašus
c.execute("SELECT * FROM sviestuvai LIMIT 10")
rezultatai = c.fetchall()

# Atspausdiname duomenis
print("\nPirmi 10 įrašų duomenų bazėje:")
for eilute in rezultatai:
    print(eilute)

# Uždaryti ryšį su duomenų baze
conn.close()


Duomenų bazėje išsaugoti 48 produktai.

Pirmi 10 įrašų duomenų bazėje:
(1, 'Šviestuvas vidaus SLV ONE TRIPLE LED, 64-65W, 4200lm, IP20 (įv.spalvos)', '1 293,41 € VNT.', '4200 lm')
(2, 'Šviestuvas lauko SLV PHOTONIA, pastatomas, E27, 60W, IP55, 90cm', '340,51 € VNT.', 'Nerasta')
(3, 'Šviestuvas Northcliffe LEVANTO PLN LED, 36W, 3650lm, IP40', '26,62 € VNT.', '3650 lm')
(4, 'Šviestuvas Northcliffe CETUS P LED, sieninis-lubinis, 12W, 1080lm, IP65', '16,34 € VNT.', '1080 lm')
(5, 'Šviestuvas vidaus SLV PANTILO CONVEX, Ø29-40cm, E27, 40W, IP20', '283,54 € VNT.', 'Nerasta')
(6, 'Šviestuvas Northcliffe DISC LED, 100-200W, 14000-28000lm, IP65', '94,38 € VNT.', '14000 lm')
(7, 'Šviestuvas lauko SLV ROTOBALL FLOOR, E27, 24W, IP44, 50cm', '279,66 € VNT.', 'Nerasta')
(8, 'Toršeras lauko SLV ADEGAN MANILA SL, nešiojamas, E27, 15W, IP44, 175.5cm', '469,98 € VNT.', 'Nerasta')
(9, 'Šviestuvas Northcliffe LIBRA LED, evakuacinis, sieninis, 1W, 6500K, IP65', '27,83 € VNT.', 'Nerasta')
(10, 'Šviestuvas SL

In [19]:
import sqlite3

# Prisijungiame prie SQLite duomenų bazės
conn = sqlite3.connect("sviestuvai.db")
c = conn.cursor()

# Pridedame trūkstamą stulpelį, jei jo nėra
try:
    c.execute("ALTER TABLE sviestuvai ADD COLUMN sviesos_srautas TEXT")
    print("Stulpelis 'sviesos_srautas' pridėtas į duomenų bazę.")
except sqlite3.OperationalError:
    print("Stulpelis 'sviesos_srautas' jau egzistuoja.")

# Išsaugome pakeitimus ir uždarome ryšį
conn.commit()
conn.close()


Stulpelis 'sviesos_srautas' pridėtas į duomenų bazę.
